### Q1. 고객 속성(id, 성별, 연령, 거주지역) 데이터와 거래 데이터를 통합한 데이터 프레임 df를 생성하세요.

고객별 속성 데이터 

- id : 1번 ~ 100번 일련번호
- gender : 0,1 정수 난수 생성
- age : 10 - 80 사이 정수 난수를 생성하고 범주화하여 age_cat 파생
- region : 1-5 사이 정수 난수 생성

거래 데이터
 - id : 1번 ~ 100번 일련번호(관측치 1000개)
 - product : 고급제품(hpd), 일반제품(lpd)로 구분
 - price : 100-200 사이 정수 난수 생성
 - qty : 1-2 사이 정수 난수 생성
 - day : 요일
 - 2020.01.01-2021.12.31 사이의 날짜 1000개 생성
 - amount : price * qty로 산출  


In [15]:
import numpy as np
import pandas as pd 

#고객 데이터
np.random.seed(1)
no = np.arange(1,101)
l1 = pd.Series(no)
gender = np.random.randint(2, size=100)
g1 = pd.Series(gender)
age = np.random.randint(10,81, size = 100)
a1 = pd.Series(age)
region = np.random.randint(1,6, size=100)
r1 = pd.Series(region)
df1 = pd.concat([l1,g1,a1,r1], axis=1)
df1.rename(columns={0:'no',1:'gender',2:'age',3:'region'}, inplace=True)
df1.head()
#거래 데이터
# t_id = np.arange(1,101, size=1000)
# product = #hpd, lpd를 통해 고객 구매 행동 패턴 파악
# price = 
# qty
# day 
# date = date_range
# amount = price * qty

#구매금액이 증가한 고객, 구매금액이 줄어든 고객을 평가할 수 있는걸 만들어야함
#예를들어, 증가했을 때는 1, 감소했을 때는 0 , 얘를 구매증감률 예측 모델에 맞는 종속변수가 되는것
# 하지만 주제가 뭐냐에 따라 다른 것. 

,no,gender,age,region
0,1,1,47,5
1,2,1,67,3
2,3,0,48,5
3,4,0,18,3
4,5,1,42,1


In [16]:
#age_cat : age를 구매력을 고려하여 age_cat로 범주화
def age_cat(x):
    cat = ''
    if x < 20 : cat=1
    elif x < 30 : cat=2
    elif x < 40 : cat = 3
    elif x < 50 : cat=4
    elif x <60 : cat=5
    else : cat = 6
    return cat
df1['age_cat'] = df1.age.apply(lambda x : age_cat(x))
df1.drop('age',axis=1, inplace=True)
df1.head()

,no,gender,region,age_cat
0,1,1,5,4
1,2,1,3,6
2,3,0,5,4
3,4,0,3,1
4,5,1,1,4


In [17]:
data = np.zeros((1000,6))
df2 = pd.DataFrame(data, columns=['no','product','price','qty','day','date'])
df2.head()

,no,product,price,qty,day,date
0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
np.random.seed(1)
df2['no'] = np.random.randint(1,101,1000)
df2['product'] = np.random.randint(1,3,1000)
df2['price']=np.random.randint(100,201,1000)
df2['qty'] = np.random.randint(1,3,1000)
df2['day'] = np.random.choice(['MON','TUE','WED','THU','FRI','SAT','SUN'],1000)
df2['date'] = pd.date_range(start='2020-01-01', end='2021-12-31', periods=1000)
df2['year'] = df2.date.dt.year
df2 = df2.drop('date', axis=1)
df2.head()


,no,product,price,qty,day,year
0,38,2,159,2,SUN,2020
1,13,1,117,1,FRI,2020
2,73,1,122,2,WED,2020
3,10,2,154,2,SUN,2020
4,76,2,113,2,THU,2020


In [34]:
print(df2.shape)
print(df2.isnull().sum())
df2.describe()

(1000, 6)
no         0
product    0
price      0
qty        0
day        0
year       0
dtype: int64


,no,product,price,qty,year
count,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000
mean,50.074000,1.484000,151.34000,1.534000,2020.499000
std,28.805317,0.499994,29.31559,0.499092,0.500249
min,1.000000,1.000000,100.00000,1.000000,2020.000000
25%,26.000000,1.000000,126.00000,1.000000,2020.000000
50%,49.000000,1.000000,153.00000,2.000000,2020.000000
75%,76.000000,2.000000,177.25000,2.000000,2021.000000
max,100.000000,2.000000,200.00000,2.000000,2021.000000


In [47]:
#product 1 고가 ~ 2 저가
def pddiff(product, price):
    if product==1 : return price * 20
    else : return price * 10 

df2['price'] = df2.apply(lambda x : pddiff(x['product'], x['price']), axis=1)
df2.head()

,no,product,price,qty,day,year
0,38,2,1590,2,SUN,2020
1,13,1,2340,1,FRI,2020
2,73,1,2440,2,WED,2020
3,10,2,1540,2,SUN,2020
4,76,2,1130,2,THU,2020


In [48]:
#quantity 1 소량 ~ 2 대량으로 조취해주기
def qtydiff(product, qty):
    if product==2 : return qty * 10
    else : return qty 
df2['qty'] = df2.apply(lambda x : qtydiff(x['product'], x['qty']), axis=1)
df2.head()

,no,product,price,qty,day,year
0,38,2,1590,20,SUN,2020
1,13,1,2340,1,FRI,2020
2,73,1,2440,2,WED,2020
3,10,2,1540,20,SUN,2020
4,76,2,1130,20,THU,2020


In [49]:
df2['amount'] = df2.qty * df2.price
df2.head()

,no,product,price,qty,day,year,amount
0,38,2,1590,20,SUN,2020,31800
1,13,1,2340,1,FRI,2020,2340
2,73,1,2440,2,WED,2020,4880
3,10,2,1540,20,SUN,2020,30800
4,76,2,1130,20,THU,2020,22600


In [60]:
pdf = df2.copy()
pdf = pdf.drop(['qty','price'], axis=1)
pdf.info()
pdf.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   no       1000 non-null   int64 
 1   product  1000 non-null   int64 
 2   day      1000 non-null   object
 3   year     1000 non-null   int64 
 4   amount   1000 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 39.2+ KB


,no,product,day,year,amount
0,38,2,SUN,2020,31800
1,13,1,FRI,2020,2340
2,73,1,WED,2020,4880
3,10,2,SUN,2020,30800
4,76,2,THU,2020,22600


In [61]:
pdf.groupby(['product','year'])['amount'].sum()

product  year
1        2020    1199820
         2021    1226620
2        2020    5597600
         2021    5539300
Name: amount, dtype: int64

### Q2. df를 수정하여 5가지 이상의 인사이트를 포함한 데이터 셋으로 변환하세요.

In [62]:
# 코로나로 고급품의 매출은 유지된 반면 생활용품 위주로 판매 급감
def pdcut(year,product,amount):
    if (year==2021) & (product==2):
        return amount*0.5
    else : return amount
pdf['amount'] = pdf.apply(lambda x : pdcut(x['year'],x['product'],x['amount']),axis=1)
pdf.head()

,no,product,day,year,amount
0,38,2,SUN,2020,31800.0
1,13,1,FRI,2020,2340.0
2,73,1,WED,2020,4880.0
3,10,2,SUN,2020,30800.0
4,76,2,THU,2020,22600.0


In [63]:
# 일반품 구매 확인 : 위와 비교용
pdf.groupby(['product','year'])['amount'].sum()

product  year
1        2020    1199820.0
         2021    1226620.0
2        2020    5597600.0
         2021    2769650.0
Name: amount, dtype: float64

In [65]:
# 고객별 구매 증감률
print(pdf.head())
# 고객별 구매상품별 연간 구매금액 변동률 산출
pdf['yp'] = pdf[['year','product']].astype(str).apply('_'.join, axis=1)
pdf.head()

   no  product  day  year   amount
0  38        2  SUN  2020  31800.0
1  13        1  FRI  2020   2340.0
2  73        1  WED  2020   4880.0
3  10        2  SUN  2020  30800.0
4  76        2  THU  2020  22600.0


,no,product,day,year,amount,yp
0,38,2,SUN,2020,31800.0,2020_2
1,13,1,FRI,2020,2340.0,2020_1
2,73,1,WED,2020,4880.0,2020_1
3,10,2,SUN,2020,30800.0,2020_2
4,76,2,THU,2020,22600.0,2020_2


In [81]:
pdf1 = pd.pivot_table(pdf, 
    index='no', 
    columns='yp',
    values='amount',
    aggfunc ='sum')
pdf1

#이제 고객 속성 데이터와 합칠 수 있음(처음 만든 고객데이터)

yp,2020_1,2020_2,2021_1,2021_2
no,,,,
1,4680.0,103000.0,16600.0,23350.0
2,11240.0,24200.0,28140.0,25700.0
3,10800.0,61000.0,3460.0,17500.0
4,12200.0,34200.0,16240.0,18850.0
5,NaN,NaN,6140.0,39950.0
...,...,...,...,...
96,4280.0,NaN,12700.0,32000.0
97,11340.0,54900.0,26080.0,60950.0
98,3420.0,58200.0,19520.0,50200.0


In [70]:
pdf1.info() # null값은 선생님은 mean을 넣으려고 함

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 1 to 100
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   2020_1  92 non-null     float64
 1   2020_2  92 non-null     float64
 2   2021_1  98 non-null     float64
 3   2021_2  89 non-null     float64
dtypes: float64(4)
memory usage: 3.9 KB


In [82]:
pdf1['2020_1'].fillna(pdf1['2020_1'].mean(), inplace=True)
pdf1['2020_2'].fillna(pdf1['2020_2'].mean(), inplace=True)
pdf1['2021_1'].fillna(pdf1['2021_1'].mean(), inplace=True)
pdf1['2021_2'].fillna(pdf1['2021_2'].mean(), inplace=True)

pdf1.columns.name=None
pdf1.reset_index(inplace=True)
pdf1.head()

,no,2020_1,2020_2,2021_1,2021_2
0,1,4680.000000,103000.000000,16600.0,23350.0
1,2,11240.000000,24200.000000,28140.0,25700.0
2,3,10800.000000,61000.000000,3460.0,17500.0
3,4,12200.000000,34200.000000,16240.0,18850.0
4,5,13041.521739,60843.478261,6140.0,39950.0


In [83]:
pdf1['2020_1'] = (pdf1['2020_1']/pdf1['2020_1'].sum()) *100
pdf1['2020_2'] = (pdf1['2020_2']/pdf1['2020_2'].sum()) *100
pdf1['2021_1'] = (pdf1['2021_1']/pdf1['2021_1'].sum()) *100
pdf1['2021_2'] = (pdf1['2021_2']/pdf1['2021_2'].sum()) *100

In [85]:
pdf1.head()

,no,2020_1,2020_2,2021_1,2021_2
0,1,0.358854,1.692868,1.326246,0.750329
1,2,0.861863,0.397742,2.248227,0.825844
2,3,0.828124,1.002573,0.276434,0.562345
3,4,0.935474,0.562098,1.297484,0.605726
4,5,1.000000,1.000000,0.490551,1.283754


In [86]:
pdf1['hpd'] = (pdf1['2021_1'] - pdf1['2020_1'])/pdf1['2020_1']
pdf1['lpd'] = (pdf1['2021_2'] - pdf1['2020_2'])/pdf1['2020_2']
pdf2 = pdf1[['no','hpd','lpd']]
pdf2.head()

,no,hpd,lpd
0,1,2.695784,-0.556770
1,2,1.608568,1.076333
2,3,-0.666192,-0.439098
3,4,0.386981,0.077617
4,5,-0.509449,0.283754


In [112]:
df = pd.merge(df1, pdf2, on='no', how='outer')
df.head()

,no,gender,region,age_cat,hpd,lpd
0,1,1,5,4,2.695784,-0.556770
1,2,1,3,6,1.608568,1.076333
2,3,0,5,4,-0.666192,-0.439098
3,4,0,3,1,0.386981,0.077617
4,5,1,1,4,-0.509449,0.283754


In [113]:
pdf.head()

,no,product,day,year,amount,yp
0,38,2,SUN,2020,31800.0,2020_2
1,13,1,FRI,2020,2340.0,2020_1
2,73,1,WED,2020,4880.0,2020_1
3,10,2,SUN,2020,30800.0,2020_2
4,76,2,THU,2020,22600.0,2020_2


In [114]:
#종속변수 정의
df_y = pdf[['no','year','amount']]
pvt = pd.pivot_table(df_y,
    index='no',
    columns='year',
    values='amount',
    aggfunc='sum')
pvt = pvt.reset_index()
pvt.columns.name = None
pvt.head()

,no,2020,2021
0,1,107680.0,39950.0
1,2,35440.0,53840.0
2,3,71800.0,20960.0
3,4,46400.0,35090.0
4,5,NaN,46090.0


In [115]:
pvt[2020].fillna(pvt[2020].mean(), inplace=True)
pvt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   no      100 non-null    int64  
 1   2020    100 non-null    float64
 2   2021    100 non-null    float64
dtypes: float64(2), int64(1)
memory usage: 2.5 KB


In [116]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   no       100 non-null    int64  
 1   gender   100 non-null    int64  
 2   region   100 non-null    int64  
 3   age_cat  100 non-null    int64  
 4   hpd      100 non-null    float64
 5   lpd      100 non-null    float64
dtypes: float64(2), int64(4)
memory usage: 5.5 KB


In [117]:
df = pd.merge(df, pvt, on='no', how='outer')
df.head()

,no,gender,region,age_cat,hpd,lpd,2020,2021
0,1,1,5,4,2.695784,-0.556770,107680.000000,39950.0
1,2,1,3,6,1.608568,1.076333,35440.000000,53840.0
2,3,0,5,4,-0.666192,-0.439098,71800.000000,20960.0
3,4,0,3,1,0.386981,0.077617,46400.000000,35090.0
4,5,1,1,4,-0.509449,0.283754,69361.428571,46090.0


In [118]:
df['ratio'] = (df[2021]-df[2020])/df[2020]
df.head()

,no,gender,region,age_cat,hpd,lpd,2020,2021,ratio
0,1,1,5,4,2.695784,-0.556770,107680.000000,39950.0,-0.628993
1,2,1,3,6,1.608568,1.076333,35440.000000,53840.0,0.519187
2,3,0,5,4,-0.666192,-0.439098,71800.000000,20960.0,-0.708078
3,4,0,3,1,0.386981,0.077617,46400.000000,35090.0,-0.243750
4,5,1,1,4,-0.509449,0.283754,69361.428571,46090.0,-0.335510


In [120]:
#고객별 구매증감율 예측 모델을 위한 분석용 데이터셋
df1 = df.drop([2020,2021],axis=1)
df1.head()

,no,gender,region,age_cat,hpd,lpd,ratio
0,1,1,5,4,2.695784,-0.556770,-0.628993
1,2,1,3,6,1.608568,1.076333,0.519187
2,3,0,5,4,-0.666192,-0.439098,-0.708078
3,4,0,3,1,0.386981,0.077617,-0.243750
4,5,1,1,4,-0.509449,0.283754,-0.335510


In [122]:
df1[['lpd','hpd','ratio']].corr()

,lpd,hpd,ratio
lpd,1.000000,-0.003388,0.428242
hpd,-0.003388,1.000000,0.156475
ratio,0.428242,0.156475,1.000000


In [124]:
def mklable(a,b): #2020년, 2021년 증감률을 위해
    if b>=a : return 1 #증가
    else : return 0 #감소

df['target'] = df.apply(lambda x : mklable(x[2020],x[2021]), axis=1)
df.head()

,no,gender,region,age_cat,hpd,lpd,2020,2021,ratio,target
0,1,1,5,4,2.695784,-0.556770,107680.000000,39950.0,-0.628993,0
1,2,1,3,6,1.608568,1.076333,35440.000000,53840.0,0.519187,1
2,3,0,5,4,-0.666192,-0.439098,71800.000000,20960.0,-0.708078,0
3,4,0,3,1,0.386981,0.077617,46400.000000,35090.0,-0.243750,0
4,5,1,1,4,-0.509449,0.283754,69361.428571,46090.0,-0.335510,0


In [125]:
y = df['target']
x = df.drop(['no',2020,2021,'ratio','target'], axis=1)

In [128]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

# 독립변수 정규화
# x_t_df = preprocessing.StandardScaler().fit(x_t_df).transform(x_t_df)

#fit = 정규화를 위해 평균, 표준편차를 계산하는 작업, transform = 정규화 작업

#학습용 데이터와 평가용 데이터를 8:2 혹은 7:3으로 분리
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 11)

print(x_train.shape)
print(x_test.shape)

(80, 5)
(20, 5)


In [129]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression #로지스틱 회귀분석
from sklearn.ensemble import RandomForestClassifier #랜덤 포레스트

#랜덤 포레스트
rf_model = RandomForestClassifier()
rf_model.fit(x_train, y_train)
rf_pred = rf_model.predict(x_test)
accuracy_rf = accuracy_score(y_test, rf_pred).round(2)

#로지스틱 회귀분석
lr_model = LogisticRegression()
lr_model.fit(x_train, y_train)
lr_pred = lr_model.predict(x_test)
accuracy_lr = accuracy_score(y_test, lr_pred).round(2)

print('rf 정확도 : {}, lr 정확도: {}'.format(accuracy_rf, accuracy_lr))

rf 정확도 : 0.8, lr 정확도: 0.85


### Q3. df를 탐색적 분석을 통하여 인사이트를 도출하세요.

In [130]:
# 증감율 - 지역, 성별, 나이, 비싼거/싼거, 코로나 상황(연별) 데이터셋